In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from src.models.load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton

In [2]:
access_path = './data/external/'
bal_from_nm = 'balance_20210503.dat'

In [3]:
bal_from = pd.read_csv(access_path + bal_from_nm)
price_db = PriceDB.instance().data

#### MP 발생일
- 2021.5.3
- 2021.6.2

In [4]:
# 두 MP발생일 사이 기간 설정
start_dt = '20210504'  # MP발생 다음날부터
end_dt = '20210601'    # MP발생 전날까지

range_dt_ymd = pd.date_range(start=start_dt, end=end_dt, freq='D')  # price_db 날짜 포맷
range_dt_mdy = range_dt_ymd.strftime('%-m/%-d/%Y')  # 잔고(balance) 날짜 포맷

#### 첫 번째 MP발생일 잔고를 해당 기간 내 매일 매일의 잔고로 복사해 두고, price 컬럼과 value 컬럼을 업데이트하면 끝.

- 잔고 복사

In [5]:
df_list = []
for cnt, dt in enumerate(range_dt_ymd):
    temp = bal_from.copy(deep=True)
    temp.date = dt
    df_list.append(temp)
    
bal_btw_rebal = pd.concat(df_list)

- `price_db`를 이용해서 Price 컬럼 업데이트.

In [6]:
bal_btw_rebal = bal_btw_rebal.set_index(['date', 'itemcode'])
price_db = price_db.set_index(['date', 'itemcode'])
bal_btw_rebal.update(price_db)
bal_btw_rebal = bal_btw_rebal.reset_index(drop=False)

- 사용자별/날짜별로 비중(`wt`) 계산

In [7]:
bal_btw_rebal.value = bal_btw_rebal.quantity*bal_btw_rebal.price
bal_btw_rebal.original = ''
bal_btw_rebal.wt = bal_btw_rebal.value/bal_btw_rebal.groupby(['date', 'userid'])['value'].transform('sum')

In [8]:
bal_btw_rebal

,date,itemcode,userid,name,asset_class,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,original
0,2021-05-04,A114260,A001,안정추구형소규모,Fixed Income,KODEX 국고채3년,2,58040,116080,58015.0,116030.0,0.165784,NaN,
1,2021-05-04,A122260,A001,안정추구형소규모,Fixed Income,KOSEF 통안채1년,2,101395,202790,101305.0,202610.0,0.289490,NaN,
2,2021-05-04,A130730,A001,안정추구형소규모,Fixed Income,KOSEF 단기자금,2,101110,202220,101100.0,202200.0,0.288905,NaN,
3,2021-05-04,A196230,A001,안정추구형소규모,Fixed Income,KBSTAR 단기통안채,1,104430,104430,104435.0,104435.0,0.149217,NaN,
4,2021-05-04,A069500,A001,안정추구형소규모,Equity,KODEX 200,1,42295,42295,42380.0,42380.0,0.060553,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2170,2021-06-01,A278540,A009,적극투자형대규모,Equity,KODEX MSCI Korea TR,23,13695,314985,13735.0,315905.0,0.240094,NaN,
2171,2021-06-01,A266370,A009,적극투자형대규모,Equity,KODEX IT,8,20760,166080,20325.0,162600.0,0.123579,NaN,
2172,2021-06-01,A292150,A009,적극투자형대규모,Equity,TIGER TOP10,7,14305,100135,14360.0,100520.0,0.076397,NaN,
2173,2021-06-01,A114260,A009,적극투자형대규모,Fixed Income,KODEX 국고채3년,2,58040,116080,57990.0,115980.0,0.088147,NaN,


In [13]:
bal_btw_rebal.to_csv('bal_btw_rebal.csv', index=False, encoding='utf-8-sig')